# Decoratoren vertiefen (Funktions Dekoratoren)
<h5>
Jetzt wollen wir uns mit den Funktions Dekoratoren beschäftigen.
Sie sind flexibler als die Klassen Dekoratoren, da wir die Funktionen direkt dekorieren können.
</h5>
<p>Als beispiel wollen wir wieder Primzahlen ermitteln und die werte zusammen addieren wie im vorherigen Kapitel nur das wir dieses mal Funktionen und keine Klassen verwenden werden</p>

<p>Wir definieren nun die Funktionen, die wir später benötigen wollen</p>

```python
def is_prime(number: int) -> bool:
    if number < 2:
        return False
    for element in range(2, int(sqrt(number)) + 1):
        if number % element == 0:
            return False
    return True

def count_prime_numbers(upper_bound: int) -> int:  # This is the function to be decorated
    result = 0
    for number in range(upper_bound):
        if is_prime(number):
            result += number
    return result
```

Wie wir aus dem Kapitel <a>[Dekoratoren](Dekoratoren.ipynb)</a> wissen, ist ein Funktions Dekorator eine Funktion, die eine andere Funktion als Argument nimmt und eine wiederum eine Funktion zurück gibt. Die Funktion, die als Argument übergeben wird, wird in der Regel in der inneren Funktion (Wrapper) aufgerufen. Die inneren Funktion kann auch Argumente an die übergebene Funktion übergeben.
 
Erstellen wir nun die Dekoratoren Benchmark und logging aus dem vorherigen Kapitel als Funktionen.

In [10]:
import logging
import sys
from time import perf_counter
from typing import Any, Callable

# Initialize logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('LOGGER_NAME')

def benchmark(func: Callable[..., Any]) -> Callable[..., Any]:  # Diese ist der eigentliche Dekorator
    def wrapper(*args: Any, **kwargs: Any) -> Any:  # Das ist die Innere-Funktion oder auch der Wrapper genannt
        start = perf_counter()
        value = func(*args, **kwargs) # Hier wird die eigentliche Funktion aufgerufen
        end = perf_counter()
        logging.info(f"Der Aufruf von {func.__name__} dauerte {end - start:.2f} Sekunden")
        return value # Rückgabe des Ergebnisses der eigentlichen Funktion

    return wrapper # Rückgabe der Innere-Funktion

def with_logging(logger: logging.Logger) -> Callable[..., Any]: # Diese ist die Dekorator-Fabrik
    def decorator(func: Callable[..., Any]) -> Callable[..., Any]: # Diese ist der eigentliche Dekorator
        def wrapper(*args: Any, **kwargs: Any) -> Any: # Das ist die Innere-Funktion oder auch der Wrapper genannt
            logger.info(f"Rufe {func.__name__} auf")
            value = func(*args, **kwargs) # Hier wird die eigentliche Funktion aufgerufen
            logger.info(f"Aufruf von {func.__name__} beendet")
            return value # Rückgabe des Ergebnisses der eigentlichen Funktion

        return wrapper # Rückgabe der Innere-Funktion
    return decorator # Rückgabe des Dekorators

Nun können wir die Funktionen dekorieren und die Dekoratoren ausführen.

In [11]:
from math import sqrt


def is_prime(number: int) -> bool:
    if number < 2:
        return False
    for element in range(2, int(sqrt(number)) + 1):
        if number % element == 0:
            return False
    return True

@with_logging(logger) 
@benchmark
def count_prime_numbers(upper_bound: int) -> int:  # Die zu dekorierende Funktion
    result = 0
    for number in range(upper_bound):
        if is_prime(number):
            result += number
    return result

logger.info(f"Result: {count_prime_numbers(1000000)}")

INFO:LOGGER_NAME:Rufe wrapper auf


INFO:root:Der Aufruf von count_prime_numbers dauerte 2.74 Sekunden
INFO:LOGGER_NAME:Aufruf von wrapper beendet
INFO:LOGGER_NAME:Result: 37550402023


Wir sehen in der Ausgabe, dass die Dekoratoren in der richtigen Reihenfolge ausgeführt werden. Wir sehen aber auch in der Ausgabe das wir als funktionsname "wrapper" ausgegeben bekommen. Das ist nicht sehr aussagekräftig. Wir können das aber ändern indem wir den Dekorator aus den in Python integrierten "functools" verwenden. Dieser Dekorator kopiert die Eigenschaften der dekorierten Funktion auf die Wrapper Funktion. Wir können also den Namen der dekorierten Funktion auf die Wrapper Funktion kopieren.

Zudem schauen wir uns noch die Dekorator Fabrik an. Diese Funktion nimmt eine Funktion als Argument und gibt eine Funktion zurück. Diese Funktion kann dann als Dekorator verwendet werden. Wir können also die Dekoratoren als Funktionen definieren und dann später als Dekoratoren verwenden.


In [12]:
import functools
import logging
import sys
from time import perf_counter
from typing import Any, Callable

# Initialize logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('LOGGER_NAME')

def benchmark(func: Callable[..., Any]) -> Callable[..., Any]:  
    @functools.wraps(func)  # Wird benötigt um den Namen der ursprünglichen Funktion zu erhalten
    def wrapper(*args: Any, **kwargs: Any) -> Any:  
        start = perf_counter()
        value = func(*args, **kwargs) 
        end = perf_counter()
        logging.info(f"Dauer des Aufrufs von {func.__name__} war {end - start:.2f} Sekunden")
        return value 

    return wrapper 

def with_logging(logger: logging.Logger) -> Callable[..., Any]: # Diese ist die Dekorator-Fabrik
    def decorator(func: Callable[..., Any]) -> Callable[..., Any]: # Diese ist der eigentliche Dekorator
        @functools.wraps(func) # Wird benötigt um den Namen der ursprünglichen Funktion zu erhalten
        def wrapper(*args: Any, **kwargs: Any) -> Any:
            logger.info(f"Calling {func.__name__}")
            value = func(*args, **kwargs)
            logger.info(f"Finished calling {func.__name__}")
            return value

        return wrapper # Rückgabe der Innere-Funktion

    return decorator # Rückgabe des Dekorators



In [13]:
@with_logging(logger) 
@benchmark
def count_prime_numbers(upper_bound: int) -> int:  # Die zu dekorierende Funktion
    result = 0
    for number in range(upper_bound):
        if is_prime(number):
            result += number
    return result

logger.info(f"Result: {count_prime_numbers(1000000)}")

INFO:LOGGER_NAME:Calling count_prime_numbers
INFO:root:Dauer des Aufrufs von count_prime_numbers war 2.63 Sekunden
INFO:LOGGER_NAME:Finished calling count_prime_numbers
INFO:LOGGER_NAME:Result: 37550402023


Wir übergeben dem Dekorator "with_logging" den logger als Argument. Dies wollen wir aber unter umständen nicht immer machen. Mithilfe von "functools.partial" können wir die Argumente des Dekorators vorab festlegen. Wir können also einen Teil der Argumente festlegen und den Rest erst später übergeben.

In [14]:
import functools
import logging
import sys
from time import perf_counter
from typing import Any, Callable

# Initialize logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('LOGGER_NAME')

def benchmark(func: Callable[..., Any]) -> Callable[..., Any]:  
    @functools.wraps(func)  # Wird benötigt um den Namen der ursprünglichen Funktion zu erhalten
    def wrapper(*args: Any, **kwargs: Any) -> Any:  
        start = perf_counter()
        value = func(*args, **kwargs) 
        end = perf_counter()
        logging.info(f"Dauer des Aufrufs von {func.__name__} war {end - start:.2f} Sekunden")
        return value 

    return wrapper 

def with_logging(func: Callable[..., Any], logger: logging.Logger) -> Callable[..., Any]: # Diese ist der eigentliche Dekorator
    @functools.wraps(func) # Wird benötigt um den Namen der ursprünglichen Funktion zu erhalten
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        logger.info(f"Calling {func.__name__}")
        value = func(*args, **kwargs)
        logger.info(f"Finished calling {func.__name__}")
        return value

    return wrapper # Rückgabe der Innere-Funktion

with_default_logging = functools.partial(with_logging, logger=logger) # Dekorator-Fabrik mit default-Argumenten


Nun nutzen wir "with_default_logging" als Dekorator. Dieser Dekorator legt den logger fest und wir müssen diesen nicht mehr übergeben. Wir müssen aber beachten, das es für die übersichtlichkeit besser ist, wenn wir dem Dekorator den logger als Argument übergeben, denn wir sehen dann schnell was dieser Dekorator macht.

Denn je Komplexer die Dekoratoren werden, desto unübersichtlicher wird es, wenn wir die Argumente nicht übergeben.

In [15]:
@with_default_logging 
@benchmark
def count_prime_numbers(upper_bound: int) -> int:  # Die zu dekorierende Funktion
    result = 0
    for number in range(upper_bound):
        if is_prime(number):
            result += number
    return result

logger.info(f"Result: {count_prime_numbers(1000000)}")

INFO:LOGGER_NAME:Calling count_prime_numbers
INFO:root:Dauer des Aufrufs von count_prime_numbers war 2.54 Sekunden
INFO:LOGGER_NAME:Finished calling count_prime_numbers
INFO:LOGGER_NAME:Result: 37550402023
